

## Chapter 8: Error Handling and Validation

In the previous chapter, we built the engine of our GraphQL server. However, a robust engine needs safety mechanisms. In a REST API, errors are often communicated via HTTP status codes (404 Not Found, 500 Server Error). GraphQL takes a different approach, treating errors as part of the response data structure.

This chapter focuses on how to effectively manage errors and validate inputs in a GraphQL environment to ensure a stable and predictable API.

### 8.1 GraphQL Error Handling Philosophy

The most fundamental shift when moving to GraphQL is understanding the HTTP status code. In a standard GraphQL setup, even if your database explodes or a user isn't found, the HTTP response will almost always be **200 OK**.

Why? Because GraphQL is a protocol that operates over a single endpoint. It returns a JSON payload that contains two top-level keys: `data` and `errors`.

*   **`data`**: Contains the result of the query (can be `null` if an error occurred).
*   **`errors`**: An array of error objects describing what went wrong.

**The Goal:** We want to move away from "throwing" generic errors and catching them globally, and instead, design our schema to represent failure states explicitly where possible, while using the `errors` array for unexpected system failures.

### 8.2 Syntax Errors vs. Validation Errors vs. Resolver Errors

Errors in GraphQL generally fall into three categories, handled at different stages of the request lifecycle.

#### 1. Syntax Errors
These occur when the query string itself is malformed (e.g., missing a curly brace, invalid syntax).
*   **Result:** The server returns a 400 Bad Request (in some implementations) or a 200 OK with an `errors` array, but **no `data` key**.
*   **Handling:** These are handled automatically by the GraphQL engine (Apollo Server, etc.). You generally do not need to write code for these.

#### 2. Validation Errors
These occur when the query syntax is valid, but it doesn't match the schema (e.g., querying a field that doesn't exist, passing a String to an Int argument).
*   **Result:** The request fails before any resolver runs.
*   **Handling:** Also handled automatically by the GraphQL engine.

#### 3. Resolver Errors (Runtime Errors)
These occur *inside* your resolver code. This is where you, the developer, have the most control. This includes "Domain Errors" like "User not found" or "Insufficient permissions."

### 8.3 Custom Errors and Error Extensions

By default, if you throw an error in a resolver, GraphQL wraps it in a generic error object.

**Example:**
```javascript
Query: {
  user: () => {
    throw new Error("Database connection failed!");
  }
}
```

**Response:**
```json
{
  "errors": [
    {
      "message": "Database connection failed!",
      "locations": [{ "line": 2, "column": 3 }],
      "path": ["user"]
    }
  ],
  "data": {
    "user": null
  }
}
```

While this works, it lacks granularity for the client. To provide more context (like error codes), we use **Error Extensions**.

**Creating a Custom Error Class:**
It is best practice to define your own error classes that extend Apollo's `ApolloError`.

```javascript
const { ApolloError } = require('apollo-server');

class UserInputError extends ApolloError {
  constructor(message, argumentName) {
    super(message, 'USER_INPUT_ERROR');
    
    // Adding custom extensions
    this.extensions = {
      code: 'BAD_USER_INPUT',
      argumentName: argumentName,
      timestamp: new Date().toISOString()
    };
  }
}

// Usage in Resolver
Mutation: {
  createUser: (_, { email }) => {
    if (!email.includes('@')) {
      throw new UserInputError('Invalid email format', 'email');
    }
    // ... create logic
  }
}
```

**Client Benefit:** The client can now read `error.extensions.code` to programmatically handle the error (e.g., show a specific toast message) rather than parsing the string message.

### 8.4 The "Partial Failure" Concept

One of GraphQL's most powerful—and sometimes confusing—features is partial failure. If a query requests multiple fields and one of them fails, GraphQL returns `null` for that specific field but still returns data for the others.

**Query:**
```graphql
query Dashboard {
  me { name }          # This succeeds
  notifications { id } # This fails (e.g., service down)
}
```

**Response:**
```json
{
  "data": {
    "me": { "name": "Alice" },
    "notifications": null
  },
  "errors": [
    { "message": "Notification service unavailable", "path": ["notifications"] }
  ]
}
```

**Implication:** Your client code must be prepared to handle `null` values gracefully. A common anti-pattern is assuming that if `data` exists, everything is successful. You must always check the `errors` array.

### 8.5 Input Validation (Library-based vs. Schema-based)

Validation ensures that the data entering your system is clean.

#### Schema-based Validation
GraphQL provides basic validation (Non-Null `!`, specific Scalar types). However, it cannot validate business rules (e.g., "String must be an email address").

#### Library-based Validation (Best Practice)
For complex validation, it is standard to use a validation library like `zod` or `joi` inside your resolvers or middleware.

**Using Zod:**

```javascript
const { z } = require('zod');

// Define a schema for the input
const UserSchema = z.object({
  name: z.string().min(2, "Name must be at least 2 characters"),
  email: z.string().email("Invalid email address"),
  age: z.number().min(18, "Must be 18 or older").optional()
});

Mutation: {
  createUser: (_, { input }) => {
    // Validate input against schema
    const result = UserSchema.safeParse(input);

    if (!result.success) {
      // Throw detailed errors
      throw new UserInputError("Validation failed", {
        errors: result.error.flatten() // Formats errors nicely for client
      });
    }

    // Proceed with valid data
    return db.createUser(result.data);
  }
}
```

This approach separates validation logic from business logic and provides precise error messages for each field.

### 8.6 Standardizing Error Responses for Clients

To maintain consistency across a large API, establish a standard error format. A highly recommended pattern is using **Union Types** for expected failures (as discussed in Chapter 5) and the `errors` array for unexpected system failures.

**Strategy:**
1.  **Expected Failures (User not found, Invalid Input):** Use Union Types or a standardized `UserError` interface in the schema. This allows the client to handle these gracefully in the UI logic.
2.  **Unexpected Failures (Database Crash, Timeout):** Throw generic errors that bubble up to the `errors` array.

**Example: Standardized Error Interface**

```graphql
interface Error {
  message: String!
  code: Int!
}

type UserNotFoundError implements Error {
  message: String!
  code: Int!
}

type Query {
  user(id: ID!): UserResult
}

union UserResult = User | UserNotFoundError
```

This gives the client a predictable structure: if the result is a `User`, render the profile; if it's `UserNotFoundError`, show a "User doesn't exist" page.

---

### 🚀 Next Up: Chapter 9 - Authentication and Authorization

**Summary:** We have secured our API against bad data; now we must secure it against bad actors. In Chapter 9, we will explore the critical topics of Authentication (who are you?) and Authorization (what can you do?). We will learn how to use the Context object to inject user information and implement strategies like middleware, resolver-level checks, and custom directives to protect our data.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='7. building_a_graphql_server.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='9. authentication_and_authorization.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
